In [1]:
import numpy as np
from cropmask.datasets import ImageDataset

In [2]:
dataset = ImageDataset()
dataset.load_imagery(dataset_dir="/mnt/azureml-filestore-896933ab-f4fd-42b2-a154-0abb35dfb0b0/landsat-1024-cp/", subset="train", image_source="landsat", class_name="agriculture", train_test_split_dir="/mnt/azureml-filestore-896933ab-f4fd-42b2-a154-0abb35dfb0b0/results/landsat-1024-cp/")

In [3]:
dataset.prepare()

In [4]:
dataset.image_ids

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107])

In [5]:
len(dataset.image_info)

108

In [7]:
for i,j in enumerate(dataset.image_info):
    if j['id'] == "tile_3584-3584":
        print(i)
        print(j)

69
{'id': 'tile_3584-3584', 'source': 'landsat', 'path': '/mnt/azureml-filestore-896933ab-f4fd-42b2-a154-0abb35dfb0b0/landsat-1024-cp/train/tile_3584-3584/image/tile_3584-3584.tif'}


In [8]:
mask, class_ids = dataset.load_mask(32)

In [9]:
class_ids.shape

(512,)

In [10]:
_idx = np.sum(mask, axis=(0, 1)) > 0

In [11]:
_idx

False

In [12]:
mask = mask.astype(bool)
mask = mask[:, :, _idx]

In [13]:
class_ids = class_ids[_idx]

In [14]:
class_ids

array([], shape=(0, 512), dtype=int32)

In [ ]:
import random
import os
import shutil
import copy
from skimage import measure
from skimage import morphology as skim
import skimage.io as skio
import warnings
import pandas as pd
import numpy as np
import pathlib
import geopandas as gpd
from rasterio import features, coords
from rasterio.plot import reshape_as_raster
import rasterio
from shapely.geometry import shape
from osgeo import gdal
from itertools import product
from rasterio import windows
from multiprocessing.pool import ThreadPool
from cropmask.label_prep import rio_bbox_to_polygon
from cropmask.misc import parse_yaml, make_dirs
from cropmask.grid import *

In [ ]:
import cropmask.preprocess as pp
from cropmask.misc import make_dirs, remove_dirs
wflow = pp.PreprocessWorkflow("/home/ryan/work/CropMask_RCNN/cropmask/preprocess_config.yaml", 
                                 "/mnt/point/western_nebraska_landsat_scenes/LT050320312005082801T1-SC20190418222350/",
                                 "/mnt/point/external/nebraska_pivots_projected.geojson")

In [ ]:
directory_list = wflow.setup_dirs()

band_list = wflow.yaml_to_band_index()

product_list = wflow.get_product_paths(band_list)

stacked_arr = wflow.load_and_stack_bands(product_list)

wflow.stack_and_save_bands()

wflow.negative_buffer_and_small_filter(-31, 100)

In [ ]:
from cropmask.sequential_grid import grid_images_rasterio_sequential

In [ ]:
import time
start = time.time()
grid_images_rasterio_sequential('/mnt/point/landsat-1024-cp/stacked/LT05_L1TP_032031_20050828_20160912_01_T1_sr.tif', '/home/ryan/work/tmp/landsat_grid_imgs')
stop = time.time()
print(stop-start)